# jsonl을 csv로 변환

In [ ]:
import json
import pandas as pd
import os

data_folder = "../../data/"  
jsonl_files = ["total_kor_counsel_bot.jsonl", "total_kor_multiturn_counsel_bot.jsonl"]  
csv_files = ["total_kor_counsel_bot.csv", "total_kor_multiturn_counsel_bot.csv"]  

# JSONL → CSV 변환
for i in range(len(jsonl_files)):  # zip() 대신 index 기반 접근
    jsonl_path = os.path.join(data_folder, jsonl_files[i])
    csv_path = os.path.join(data_folder, csv_files[i])

    data = []
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))

    df = pd.DataFrame(data)

    df.to_csv(csv_path, index=False, encoding="utf-8")

    print(f"✅ {jsonl_files[i]} → {csv_files[i]} 변환 완료! (저장 경로: {csv_path})")


✅ total_kor_counsel_bot.jsonl → total_kor_counsel_bot.csv 변환 완료! (저장 경로: ../data/total_kor_counsel_bot.csv)
✅ total_kor_multiturn_counsel_bot.jsonl → total_kor_multiturn_counsel_bot.csv 변환 완료! (저장 경로: ../data/total_kor_multiturn_counsel_bot.csv)


# total_kor_counsel_bot.csv 분석

- \n을 공백으로 변경
- 2개 이상의 연속 공백 단일 공백으로 변환하고 앞뒤 공백 제거
- 특수문자는 있긴하지만 제외는 하지 않음 -> 대화중에 쓰는 경우 고려
- 완전히 중복된 데이터 제거

In [ ]:
import pandas as pd

csv_file = "../../data/total_kor_counsel_bot.csv" 

In [26]:
df = pd.read_csv(csv_file)

num_rows = len(df)
num_columns = len(df.columns)

print(f"총 데이터 개수: {num_rows}개")
print(f"총 컬럼 개수: {num_columns}개")
print("\n 데이터 컬럼명:")
print(df.columns)
print(df.head())

총 데이터 개수: 13234개
총 컬럼 개수: 2개

 데이터 컬럼명:
Index(['input', 'output'], dtype='object')
                                               input  \
0  저 사실, 약간 중2병 같은 걸 증상을 보이고 있습니다.\n그래서 대학교를 가면서도...   
1  회사에서 일이 너무 바빠서 스트레스 받아요.\n회사에서 일이 너무 많아서 스트레스 ...   
2  남들에게 이상해보인다는 생각 때문에 항상 불안하고 스트레스 받아요. 눈치보지 말라고...   
3  요즘 회사에서 일이 많아지면서 나 혼자 다 이뤄내야 한다는 압박감이 들어요. 이전에...   
4  저는 요즘 일이 많아지면서 체력적으로도, 정신적으로도 지치게 됐어요. 마감기한이 얼...   

                                              output  
0  사우님이 중2병적인 행동과 태도를 가지고 있다고 느껴져 스트레스를 받으시는 상황입니...  
1  사우님이 회사에서 일이 너무 많아서 지쳐서 스트레스를 받고 있으시다는데요. 하루종일...  
2  사우님 남들에게 이상하게 보일까봐 불안하고 스트레스 받으시네요. 눈치보지 말라고 해...  
3  사우님은 회사에서 일이 많아져서 혼자 다 처리해야 하는 압박감 때문에 불안감을 느끼...  
4  사우님 요즘 일이 많아지면서 체력적으로도, 정신적으로도 지친 상태이고 일 할 때마다...  


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13234 entries, 0 to 13233
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   input   13234 non-null  object
 1   output  13234 non-null  object
dtypes: object(2)
memory usage: 206.9+ KB


In [28]:
# input 컬럼에서 \n 포함된 개수
num_input_newline = df["input"].astype(str).str.contains("\n").sum()

# output 컬럼에서 \n 포함된 개수
num_output_newline = df["output"].astype(str).str.contains("\n").sum()

print(f"input 컬럼에서 \\n 포함된 개수: {num_input_newline}개")
print(f"output 컬럼에서 \\n 포함된 개수: {num_output_newline}개")


input 컬럼에서 \n 포함된 개수: 6314개
output 컬럼에서 \n 포함된 개수: 13234개


In [29]:
# \n을 공백으로 변경 -> 한 줄로 만들기
df["input"] = df["input"].str.replace("\n", " ", regex=True)
df["output"] = df["output"].str.replace("\n", " ", regex=True)

print(f"\\n 제거 완료!")

\n 제거 완료!


In [31]:
# input 컬럼에서 \n 포함된 개수
num_input_newline = df["input"].astype(str).str.contains("\n").sum()

# output 컬럼에서 \n 포함된 개수
num_output_newline = df["output"].astype(str).str.contains("\n").sum()

print(f"input 컬럼에서 \\n 포함된 개수: {num_input_newline}개")
print(f"output 컬럼에서 \\n 포함된 개수: {num_output_newline}개")

input 컬럼에서 \n 포함된 개수: 0개
output 컬럼에서 \n 포함된 개수: 0개


In [33]:
#  2개 이상의 연속 공백 개수 확인 (\s+ 패턴 존재 여부)
num_input_spaces = df["input"].astype(str).str.contains(r"\s{2,}").sum()
num_output_spaces = df["output"].astype(str).str.contains(r"\s{2,}").sum()

print(f"input 컬럼에서 연속된 공백 포함된 개수: {num_input_spaces}개")
print(f"output 컬럼에서 연속된 공백 포함된 개수: {num_output_spaces}개")

input 컬럼에서 연속된 공백 포함된 개수: 1017개
output 컬럼에서 연속된 공백 포함된 개수: 10785개


In [ ]:
# 연속된 공백을 단일 공백으로 변환하고 앞뒤 공백 제거
df["input"] = df["input"].str.replace(r"\s+", " ", regex=True).str.strip()
df["output"] = df["output"].str.replace(r"\s+", " ", regex=True).str.strip()

num_input_spaces = df["input"].astype(str).str.contains(r"\s{2,}").sum()
num_output_spaces = df["output"].astype(str).str.contains(r"\s{2,}").sum()

print(f"input 컬럼에서 연속된 공백 포함된 개수: {num_input_spaces}개")
print(f"output 컬럼에서 연속된 공백 포함된 개수: {num_output_spaces}개")

input 컬럼에서 연속된 공백 포함된 개수: 0개
output 컬럼에서 연속된 공백 포함된 개수: 0개


In [21]:
import re

# HTML 태그 포함 여부 확인
num_input_html = df["input"].astype(str).str.contains(r"<[^>]+>").sum()
num_output_html = df["output"].astype(str).str.contains(r"<[^>]+>").sum()

print(f"input 컬럼에서 HTML 태그 포함 개수: {num_input_html}개")
print(f"output 컬럼에서 HTML 태그 포함 개수: {num_output_html}개")

# 이상한 특수문자 포함 여부 확인
num_input_special = df["input"].astype(str).str.contains(r"[^가-힣a-zA-Z0-9\s.,!?]").sum()
num_output_special = df["output"].astype(str).str.contains(r"[^가-힣a-zA-Z0-9\s.,!?]").sum()

print(f"input 컬럼에서 이상한 특수문자 포함 개수: {num_input_special}개")
print(f"output 컬럼에서 이상한 특수문자 포함 개수: {num_output_special}개")

input 컬럼에서 HTML 태그 포함 개수: 0개
output 컬럼에서 HTML 태그 포함 개수: 0개
input 컬럼에서 이상한 특수문자 포함 개수: 642개
output 컬럼에서 이상한 특수문자 포함 개수: 10058개


In [22]:
num_duplicates = df.duplicated(subset=["input", "output"]).sum()
print(f"중복된 질문-답변 개수: {num_duplicates}개")

중복된 질문-답변 개수: 30개


In [38]:
# 중복된 질문-답변 데이터 추출
duplicate_rows = df[df.duplicated(subset=["input", "output"], keep=False)]

print(duplicate_rows.head(30))

                                                   input  \
172    실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
216    실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
784    실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
1505   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
1639   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
1876   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
2471   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
2483   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
3185   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
3457   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
3642   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
4596   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
5795   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
6395   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
6527   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...   
6780   실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어

In [ ]:
# CSV로 저장
duplicate_rows.to_csv("../../data/duplicates_review.csv", index=False, encoding="utf-8")

print("중복된 질문-답변 30개를 duplicates_review.csv 파일로 저장했습니다.")

중복된 질문-답변 30개를 duplicates_review.csv 파일로 저장했습니다.


In [ ]:
# 중복된 질문-답변 데이터 찾기 (keep=False: 모든 중복된 행을 표시)
duplicate_indices = df[df.duplicated(subset=["input", "output"], keep=False)].index

# 앞뒤 행 포함해서 출력
for idx in duplicate_indices:
    print("\n중복 데이터 전후 문맥 확인")
    
    if idx > 0:
        print(f"\n이전 데이터 (index {idx-1}):\n{df.iloc[idx-1]}\n")
    print(f"\n중복된 데이터 (index {idx}):\n{df.iloc[idx]}\n")

    if idx < len(df) - 1:
        print(f"\n다음 데이터 (index {idx+1}):\n{df.iloc[idx+1]}\n")
    print("-" * 80)


중복 데이터 전후 문맥 확인

이전 데이터 (index 171):
input     저는 회사에서 매니저님과 서로 다른 의견을 제시하게 되면 자신의 생각을 피력하는 것...
output    사우님 매니저님과 다른 의견을 제시하면 자신의 생각을 말하기 어렵다고 하시네요. 매...
Name: 171, dtype: object


중복된 데이터 (index 172):
input     실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...
output    사우님 시험을 잘보지 못할까 걱정되는 마음에 시험에서 실수하면 안된다고 생각하시네요...
Name: 172, dtype: object


다음 데이터 (index 173):
input     요즘 집에서 일하니까 매일 같이 먹고 자는 가족들 때문에 집안 일에 불만이 많이 생...
output    사우님이 요즘 집에서 일하면서 가족들과 함께 지내다보니 가족들이 자신들이 하는 일에...
Name: 173, dtype: object

--------------------------------------------------------------------------------

중복 데이터 전후 문맥 확인

이전 데이터 (index 215):
input     요즘 내가 이상한 것 같아요. 학교에 가는 것도 싫고, 친구들과 놀기도 귀찮고, 집...
output    사우님 요즘은 학교에 가는 것도 싫고, 친구들과 놀기도 귀찮고, 집에서 아무것도 하...
Name: 215, dtype: object


중복된 데이터 (index 216):
input     실수를 너무 많이해요. 저는 시험이라는 글자가 나오고나 들어도 속이 울렁거리고 기분...
output    사우님 시험을 잘보지 못할까 걱정되는 마음에 시험에서 실수하면 안된다고 생각하시네요...
Name: 216, dtype: object


다음 데이터 (index 217):
input     업무가 지금 너무 바쁘게 진행되

In [ ]:
# 중복된 질문-답변 데이터 찾기 (keep=False: 모든 중복된 행 표시)
duplicate_indices = df[df.duplicated(subset=["input", "output"], keep=False)].index

# 앞뒤 행 포함한 데이터 저장을 위한 리스트
review_data = []

for idx in duplicate_indices:
    row = {
        "index": idx,
        "prev_input": df.iloc[idx - 1]["input"] if idx > 0 else "",
        "prev_output": df.iloc[idx - 1]["output"] if idx > 0 else "",
        "duplicate_input": df.iloc[idx]["input"],
        "duplicate_output": df.iloc[idx]["output"],
        "next_input": df.iloc[idx + 1]["input"] if idx < len(df) - 1 else "",
        "next_output": df.iloc[idx + 1]["output"] if idx < len(df) - 1 else "",
    }
    review_data.append(row)

# 데이터프레임 변환
df_review = pd.DataFrame(review_data)

# CSV로 저장
csv_path = "../data/../duplicates_review_with_context.csv"
df_review.to_csv(csv_path, index=False, encoding="utf-8")

print(f"중복 데이터 확인용 CSV 저장 완료! (경로: {csv_path})")


중복 데이터 확인용 CSV 저장 완료! (경로: ../data/duplicates_review_with_context.csv)


In [ ]:
# 완전히 동일한 중복된 데이터 제거
df = df.drop_duplicates(subset=["input", "output"], keep="first").reset_index(drop=True)

# 중복 개수 다시 확인
num_duplicates_after = df.duplicated(subset=["input", "output"]).sum()

print(f"중복 제거 후 남은 중복된 질문-답변 개수: {num_duplicates_after}개")

중복 제거 후 남은 중복된 질문-답변 개수: 0개


In [48]:
# 질문 & 답변 길이 확인 (토큰 기준이 아니라 글자 수 기준)
df["input_length"] = df["input"].str.len()
df["output_length"] = df["output"].str.len()

# 300자 이상인 데이터 개수 확인
long_inputs = (df["input_length"] > 300).sum()
long_outputs = (df["output_length"] > 300).sum()

print(f"300자 이상 긴 질문 개수: {long_inputs}개")
print(f"300자 이상 긴 답변 개수: {long_outputs}개")

300자 이상 긴 질문 개수: 777개
300자 이상 긴 답변 개수: 13204개


In [ ]:
clean_csv_path = "../data/../total_kor_counsel_bot_clean.csv"
df.to_csv(clean_csv_path, index=False, encoding="utf-8")

print(f"데이터 저장 완료! (경로: {clean_csv_path})")

데이터 저장 완료! (경로: ../data/total_kor_counsel_bot_clean.csv)
